# Introduction

This notebook demonstrates a few ways to run backtests. 

We start with the cell magic **%use** to load the required packages. The **Welcome()** statement is optional, but gives usefull information on the environment the notebook is running in.

In [ ]:
%use @http://roboquant.org/roboquant.json
Welcome()

# Create a Roboquant instance
Besides the name of the platform, the `Roboquant` class is also the engine of the framework. It ties all the components together that make-up a back-test. It also runs the actual tests and live trading.

There many default values for the constructor of `Roboquant`, so the only two main parameters that need to be passed to get started are:

1. The strategy we want to test. For this example we use a Exponential Moving Average Crossover with its default settings. This comes out-of-the-box with roboquant.
2. Any metrics we want to captured during the run to see how our run is performing. In this case we use the AccountSummary metric.

roboquant allows for much more customization, but for this introduction notebook we'll keep it simple.  

In [ ]:
// The data feed we'll be using 
val feed = AvroFeed.sp500()

// Use the default included Exponential Moving Average Crossover strategy
val strategy = EMACrossover()

// For this notebook we'll only monitor the AccountSummary metrics (and epecially the total equity) 
val roboquant =  Roboquant(strategy, AccountSummary())

# Single Backtest
The most simple backtest approach is to run the backtest over all the data available in the feed. So there is only a single run. 

In [ ]:
roboquant.run(feed)

# Inspect Results

Likely the first thing you want to know after the run has finished is: how does the account look like? 

An account in roboquant is owned by the boker and contains the following 4 types of data: 

1. the cash balances
2. the portfolio with its positions
3. the orders, both open and closed orders
4. any trades made, so orders that have been (partially) filled


> Please note the use of the **summary()** method in the next notebook cells. Many classes in roboquant implement this method and it provides a nice summary of the state on an object, like in this case the account and portfolio object.


In [ ]:
// get a snapshot of the current state of the account
val account = roboquant.broker.account
account.summary()

In [ ]:
// You can also get more details, for example about the assets in the portfolio
account.portfolio.summary()

In [ ]:
val trades = account.trades
trades[0..10].summary()

In [ ]:
account.closedOrders.takeLast(20).summary()

# Metric Results
The account contains the state at the end of the run. Metrics are more powerful in that they capture state during each step of the run. So they give an overview how a metric evolves over time during the run. It is the logger who is responsible for storing or logging the metric results.  

In [ ]:
var logger = roboquant.logger
logger.metricNames

In [ ]:
logger.getMetric("account.equity").summary()

# More detailed inspections

After a quick scan of the results, you might want to dive into some more details. For example how much profit or loss did the strategy make for a single asset, like in this example Apple?

In [ ]:
val appleTrades = trades.filter { it.asset.symbol == "AAPL" }
appleTrades.realizedPNL

Or how much profit did we make each year?

In [ ]:
for (timeframe in trades.timeframe.split(1.years)) { 
    val pnl = trades.filter { it.time in timeframe }.realizedPNL
    println("$timeframe => profit is $pnl")
}

# Walk forward

To move from a single run to a walk forward back test is simple and requires hardly any extra code. We just run the back test multiple times with each iteration limited to a certain timeframe. 

In [ ]:
// This reset will clear any state the logger has
roboquant.reset()

feed.split(1.years).forEach { 
    roboquant.run(feed, it)   
}

In [ ]:
val logger = roboquant.logger

// print the max equity per run
for (entry in logger.getMetric("account.equity").groupBy { it.info.run }.max()) {
        println("${entry.key} = ${entry.value.value}")
}

# Random Samples

An even more thorough backtest strategy is to sample many random periods of the same length and see how they compare. It provides a good insights how returns are distributes and what best and worst possible outcomes are for the given period size.

In [ ]:
// This reset will clear any state the logger has
roboquant.reset()

repeat(25) {
    // get a random sample with 250 trading days 
    val tf = feed.sample(250) 
    roboquant.run(feed, tf) 
}

In [ ]:
val equity = logger.getMetric("account.equity")

val minEquity = equity.minByOrNull { it.value }
val maxEquity = equity.maxByOrNull { it.value }

println(minEquity) // worst case scenario
println(maxEquity) // best case scenario

# Next Steps
A great way to get insights into the performance of your strategies is by using one of the many included charts. If you want to find out more about that, check out the **charts notebook** next. 
